In [1]:
import os
import numpy as np
import qiskit
import math
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService
from dotenv import load_dotenv

load_dotenv()
IBM_API_TOKEN = os.getenv("IBM_API_TOKEN")

In [24]:
N = int(input("Ingrese un número que se pueda descomponer en 2 factores primos: "))
lim_inf = 2
lim_sup = N - 2
qubit_number = int(np.ceil(np.log2(N + 1))) if N > 0 else 1

In [ ]:
def QFT(qubit_number: int):
    """Transformada de Fourier Cuántica para N qubits."""
    circuit = QuantumCircuit(qubit_number)
    for i in range(qubit_number-1, -1, -1):
        circuit.h(i)
        for j in range(i-1, -1, -1):
            theta = math.pi / (2 ** (i - j))
            circuit.cp(theta, j, i)
    
    for i in range(qubit_number // 2):
        circuit.swap(i, qubit_number - i - 1)

    compuert = circuit.to_gate()
    compuert.name = "QFT"
    return compuert


In [2]:
# Uf implementa: |x⟩|y⟩ → |x⟩|y · a^x mod N⟩
# Donde:
# - x es el exponente (registro de control)
# - y es el resultado (registro objetivo)
# - a es la base elegida aleatoriamente
# - N es el número a factorizar

N = 15  # Ejemplo: queremos factorizar 15
a = 7   # Base elegida (debe cumplir gcd(a,N) = 1)

print(f"Ejemplo: N = {N}, a = {a}")
print(f"Uf implementa la función f(x) = {a}^x mod {N}")

# Calculemos algunos valores de f(x) manualmente
print("\nValores de f(x):")
for x in range(8):
    fx = pow(a, x, N)
    print(f"f({x}) = {a}^{x} mod {N} = {fx}")

# Encontremos el período
print(f"\nBuscando el período...")
period = None
for p in range(1, N):
    if pow(a, p, N) == 1:
        period = p
        break

print(f"Período encontrado: {period}")
print(f"Verificación: {a}^{period} mod {N} = {pow(a, period, N)}")

Ejemplo: N = 15, a = 7
Uf implementa la función f(x) = 7^x mod 15

Valores de f(x):
f(0) = 7^0 mod 15 = 1
f(1) = 7^1 mod 15 = 7
f(2) = 7^2 mod 15 = 4
f(3) = 7^3 mod 15 = 13
f(4) = 7^4 mod 15 = 1
f(5) = 7^5 mod 15 = 7
f(6) = 7^6 mod 15 = 4
f(7) = 7^7 mod 15 = 13

Buscando el período...
Período encontrado: 4
Verificación: 7^4 mod 15 = 1


In [5]:
def create_basic_Uf_structure(n_control_qubits, n_target_qubits):
    """
    Crea la estructura básica de Uf sin implementar la lógica específica.
    
    Args:
        n_control_qubits: Número de qubits para x (exponente)
        n_target_qubits: Número de qubits para el resultado
    """
    total_qubits = n_control_qubits + n_target_qubits
    qc = QuantumCircuit(total_qubits)
    qc.name = f"Uf_estructura_{n_control_qubits}_{n_target_qubits}"
    
    # Los primeros n_control_qubits representan x
    # Los siguientes n_target_qubits representan y (resultado)
    
    print(f"Circuito creado con {total_qubits} qubits:")
    print(f"  - Qubits 0-{n_control_qubits-1}: registro de control (x)")
    print(f"  - Qubits {n_control_qubits}-{total_qubits-1}: registro objetivo (y)")
    
    return qc

# Calcular qubits necesarios para N = 15
n_target_qubits = int(np.ceil(np.log2(N + 1)))  # Necesario para representar números hasta N
n_control_qubits = 4  # Para el exponente x

print(f"\nPara N = {N}:")
print(f"Qubits objetivo necesarios: {n_target_qubits}")
print(f"Qubits de control: {n_control_qubits}")

# Crear estructura básica
uf_basic = create_basic_Uf_structure(n_control_qubits, n_target_qubits)
print(f"\nCircuito básico creado: {uf_basic.name}")


Para N = 15:
Qubits objetivo necesarios: 4
Qubits de control: 4
Circuito creado con 8 qubits:
  - Qubits 0-3: registro de control (x)
  - Qubits 4-7: registro objetivo (y)

Circuito básico creado: Uf_estructura_4_4


In [7]:
def explain_binary_exponentiation(x_value, a, N):
    """
    Explica cómo funciona la exponenciación binaria.
    """
    print(f"Calculando {a}^{x_value} mod {N} usando exponenciación binaria:")
    
    # Convertir x a binario
    binary_x = bin(x_value)[2:]  # Quitar el '0b'
    print(f"{x_value} en binario = {binary_x}")
    
    # Descomponer la exponenciación
    terms = []
    result = 1
    
    for i, bit in enumerate(reversed(binary_x)):
        power_of_2 = 2**i
        if bit == '1':
            base_power = pow(a, power_of_2, N)
            terms.append(f"{a}^{power_of_2}")
            result = (result * base_power) % N
            print(f"  Bit {i}: {a}^{power_of_2} mod {N} = {base_power}")
    
    print(f"Entonces: {a}^{x_value} = {' × '.join(terms)} mod {N}")
    print(f"Resultado: {result}")
    print(f"Verificación directa: {pow(a, x_value, N)}")
    
    return result

# Ejemplo práctico
x_ejemplo = 5
print(f"Ejemplo con x = {x_ejemplo}:")
explain_binary_exponentiation(x_ejemplo, a, N)

print(f"\nEsto significa que para implementar Uf cuánticamente:")
print(f"1. Cada qubit de control corresponde a un bit de x")
print(f"2. Si el qubit i está en |1⟩, multiplicamos por a^(2^i) mod N")
print(f"3. Usamos multiplicación modular controlada")


Ejemplo con x = 5:
Calculando 7^5 mod 15 usando exponenciación binaria:
5 en binario = 101
  Bit 0: 7^1 mod 15 = 7
  Bit 2: 7^4 mod 15 = 1
Entonces: 7^5 = 7^1 × 7^4 mod 15
Resultado: 7
Verificación directa: 7

Esto significa que para implementar Uf cuánticamente:
1. Cada qubit de control corresponde a un bit de x
2. Si el qubit i está en |1⟩, multiplicamos por a^(2^i) mod N
3. Usamos multiplicación modular controlada


In [ ]:
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import QFT as QiskitQFT
import numpy as np
import math
import random
from math import gcd

def crear_sumador_cuantico(num_qubits):
    """
    Crea un sumador cuántico usando QFT
    Implementa |a⟩|b⟩ → |a⟩|(a+b) mod 2^n⟩
    """
    circuit = QuantumCircuit(2 * num_qubits)
    
    # Aplicar QFT al segundo registro
    for i in range(num_qubits):
        circuit.h(num_qubits + i)
        for j in range(i):
            circuit.cp(math.pi / (2**(i-j)), num_qubits + j, num_qubits + i)
    
    # Suma controlada usando rotaciones de fase
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i + j < num_qubits:
                circuit.cp(math.pi / (2**(num_qubits - 1 - i - j)), i, num_qubits + i + j)
    
    # QFT inversa al segundo registro
    for i in range(num_qubits-1, -1, -1):
        for j in range(i):
            circuit.cp(-math.pi / (2**(i-j)), num_qubits + j, num_qubits + i)
        circuit.h(num_qubits + i)
    
    return circuit

def crear_multiplicador_por_constante(a, N, num_qubits):
    """
    Crea un circuito que multiplica por una constante 'a' módulo N
    Usa el método de suma repetida optimizada
    """
    circuit = QuantumCircuit(num_qubits)
    
    # Implementación simplificada usando rotaciones controladas
    # Para cada bit de entrada, suma a*2^i si el bit está activado
    for i in range(num_qubits):
        # Calcular (a * 2^i) mod N
        multiplicando = (a * pow(2, i, N)) % N
        
        # Aplicar rotaciones que representen esta suma
        for j in range(num_qubits):
            if (multiplicando >> j) & 1:
                circuit.cp(math.pi / (2**(num_qubits-1-j)), i, j)
    
    return circuit

def crear_exponenciacion_modular_controlada(a, exponente, N, qubit_control, qubits_target):
    """
    Crea circuito para exponenciación modular controlada
    |control⟩|y⟩ → |control⟩|y * a^exponente mod N⟩
    """
    num_qubits = len(qubits_target)
    circuit = QuantumCircuit(1 + num_qubits)
    
    # Calcular a^exponente mod N
    a_power = pow(a, exponente, N)
    
    # Implementar multiplicación modular controlada
    # Usando el método de multiplicación por adición repetida
    temp_reg = list(range(1, num_qubits + 1))
    
    # Para cada bit del multiplicador
    for i in range(num_qubits):
        if (a_power >> i) & 1:
            # Suma controlada de 2^i veces el valor actual
            for j in range(num_qubits):
                if j + i < num_qubits:
                    # Triple control: control bit, input bit i, y resultado
                    circuit.ccx(0, 1 + i, 1 + j + i)
    
    return circuit

def crear_circuito_Uf_funcional(a, N, num_qubits_registro, num_qubits_trabajo):
    """
    Implementación funcional del circuito Uf usando componentes aritméticos
    """
    total_qubits = num_qubits_registro + num_qubits_trabajo
    circuit = QuantumCircuit(total_qubits)
    
    # Implementación usando el método de exponenciación por cuadrados
    # Para cada bit i del registro de control, aplicamos a^(2^i) mod N
    
    for i in range(num_qubits_registro):
        exponente = pow(2, i)  # 2^i
        a_power = pow(a, exponente, N)  # a^(2^i) mod N
        
        # Aplicar multiplicación modular controlada
        # Si el bit i está en |1⟩, multiplicar por a^(2^i)
        
        # Implementación usando operaciones XOR controladas
        for j in range(num_qubits_trabajo):
            if (a_power >> j) & 1:
                circuit.cx(i, num_qubits_registro + j)
    
    return circuit

def crear_circuito_Uf_con_ancillas(a, N, num_qubits_registro):
    """
    Implementación más robusta usando qubits auxiliares
    """
    num_qubits_trabajo = int(np.ceil(np.log2(N)))
    num_qubits_aux = num_qubits_trabajo  # Qubits auxiliares para cálculos
    
    total_qubits = num_qubits_registro + num_qubits_trabajo + num_qubits_aux
    circuit = QuantumCircuit(total_qubits)
    
    # Inicializar registro de trabajo en |1⟩
    circuit.x(num_qubits_registro)
    
    # Para cada qubit de control
    for i in range(num_qubits_registro):
        # Calcular a^(2^i) mod N
        exponente = pow(2, i)
        factor = pow(a, exponente, N)
        
        # Implementar multiplicación modular controlada usando qubits auxiliares
        inicio_trabajo = num_qubits_registro
        inicio_aux = num_qubits_registro + num_qubits_trabajo
        
        # Usar el qubit de control i para activar la multiplicación
        # Implementación simplificada: XOR controlado con el patrón del factor
        for j in range(num_qubits_trabajo):
            if (factor >> j) & 1:
                circuit.cx(i, inicio_trabajo + j)
    
    return circuit

def QFT_personalizada(num_qubits):
    """QFT personalizada optimizada"""
    circuit = QuantumCircuit(num_qubits)
    
    for i in range(num_qubits):
        circuit.h(i)
        for j in range(i + 1, num_qubits):
            circuit.cp(math.pi / (2**(j-i)), i, j)
    
    # Invertir orden de qubits
    for i in range(num_qubits // 2):
        circuit.swap(i, num_qubits - 1 - i)
    
    return circuit

def algoritmo_shor_funcional():
    """
    Versión completamente funcional del algoritmo de Shor
    """
    print("=== Algoritmo de Shor Funcional ===")
    
    # Entrada del usuario
    N = int(input("Ingrese el número a factorizar: "))
    
    if N <= 1:
        print("El número debe ser mayor que 1")
        return
    
    if N % 2 == 0:
        print(f"Factores encontrados: 2 y {N//2}")
        return
    
    # Parámetros del algoritmo
    num_qubits_N = int(np.ceil(np.log2(N + 1)))
    num_qubits_registro = 2 * num_qubits_N  # Para mejor precisión
    
    print(f"Número a factorizar: {N}")
    print(f"Qubits necesarios para N: {num_qubits_N}")
    print(f"Qubits para registro de fase: {num_qubits_registro}")
    
    max_intentos = 5
    
    for intento in range(max_intentos):
        print(f"\n--- Intento {intento + 1} ---")
        
        # Elegir 'a' aleatorio
        a = random.randint(2, N - 2)
        print(f"Base elegida: a = {a}")
        
        # Verificar coprimality
        if gcd(a, N) != 1:
            factor = gcd(a, N)
            print(f"¡Factor encontrado! gcd({a}, {N}) = {factor}")
            print(f"Factores: {factor} y {N // factor}")
            return
        
        # Construir el circuito principal
        print("Construyendo circuito cuántico...")
        
        # Registros cuánticos
        registro_fase = QuantumRegister(num_qubits_registro, 'fase')
        registro_trabajo = QuantumRegister(num_qubits_N, 'trabajo')
        mediciones = ClassicalRegister(num_qubits_registro, 'medicion')
        
        # Crear circuito principal
        qc = QuantumCircuit(registro_fase, registro_trabajo, mediciones)
        
        # Paso 1: Preparar superposición en registro de fase
        qc.h(registro_fase)
        
        # Paso 2: Inicializar registro de trabajo en |1⟩
        qc.x(registro_trabajo[0])
        
        # Paso 3: Aplicar operador Uf (versión funcional)
        print("Aplicando operador Uf...")
        
        # Implementar U^(2^j) para cada j
        for j in range(num_qubits_registro):
            exponente_total = pow(2, j)
            
            # Para cada potencia, aplicar multiplicación modular controlada
            for k in range(exponente_total):
                # Multiplicación por 'a' módulo N, controlada por qubit j
                factor_actual = pow(a, k + 1, N)
                
                # Implementar la multiplicación usando XOR controlado
                for bit in range(num_qubits_N):
                    if (factor_actual >> bit) & 1:
                        qc.cx(registro_fase[j], registro_trabajo[bit])
        
        # Paso 4: Aplicar QFT inversa al registro de fase
        print("Aplicando QFT inversa...")
        qft_inv = QFT_personalizada(num_qubits_registro).inverse()
        qc.append(qft_inv, registro_fase)
        
        # Paso 5: Medir registro de fase
        qc.measure(registro_fase, mediciones)
        
        print(f"Circuito construido:")
        print(f"- Total de qubits: {qc.num_qubits}")
        print(f"- Profundidad: {qc.depth()}")
        print(f"- Número de compuertas: {len(qc.data)}")
        
        # El circuito está listo para ejecutar
        print("✓ Circuito listo para ejecutar en IBM Quantum")
        
        # Simular localmente para verificar
        print("\nCircuito construido exitosamente.")
        print("Para ejecutar en IBM Quantum, use este objeto 'qc'")
        
        return qc  # Retornar el circuito para uso externo

# Ejecutar
if __name__ == "__main__":
    
    # O ejecutar versión completa (descomenta la siguiente línea):
    circuito_completo = algoritmo_shor_funcional()

=== Algoritmo de Shor Funcional ===
Número a factorizar: 21
Qubits necesarios para N: 5
Qubits para registro de fase: 10

--- Intento 1 ---
Base elegida: a = 13
Construyendo circuito cuántico...
Aplicando operador Uf...
Aplicando QFT inversa...
Circuito construido:
- Total de qubits: 15
- Profundidad: 2048
- Número de compuertas: 2069
✓ Circuito listo para ejecutar en IBM Quantum

Circuito construido exitosamente.
Para ejecutar en IBM Quantum, use este objeto 'qc'
